## REGENIE Unit Test Development

In [1]:
import sgkit_plink
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from sgkit.stats import regenie
from sgkit.tests import test_regenie
from sgkit.stats.association import linear_regression
from sklearn.metrics import r2_score as sk_r2
from sgkit.stats.regenie import r2_score as sg_r2
from pathlib import Path
import sgkit as sg
import yaml
%load_ext autoreload
%autoreload 2

### Load Simulated Data

In [2]:
dataset = 'sim_sm_03'
paramset = 'wgr_01'

In [3]:
dataset_dir = Path('data/dataset/' + dataset)
result_dir = Path('data/result/' + dataset + '-' + paramset)
dataset_dir, result_dir

(PosixPath('data/dataset/sim_sm_03'),
 PosixPath('data/result/sim_sm_03-wgr_01'))

In [4]:
with open('config.yml') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)
ds_config, ps_config = config['datasets'][dataset], config['paramsets'][paramset]
ds_config, ps_config

({'n_variants': 250,
  'n_samples': 50,
  'n_covars': 3,
  'n_contigs': 10,
  'n_traits': 1},
 {'variant_block_size': 10, 'sample_block_size': 10, 'alphas': [1000]})

In [5]:
ds = sgkit_plink.read_plink(str(dataset_dir / 'genotypes'), bim_int_contig=True)
ds

<xarray.Dataset>
Dimensions:             (alleles: 2, ploidy: 2, samples: 50, variants: 250)
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    variant/contig      (variants) int16 dask.array<chunksize=(250,), meta=np.ndarray>
    variant/position    (variants) int32 dask.array<chunksize=(250,), meta=np.ndarray>
    variant/alleles     (variants, alleles) |S1 dask.array<chunksize=(250, 1), meta=np.ndarray>
    sample/id           (samples) <U1 dask.array<chunksize=(50,), meta=np.ndarray>
    call/genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(250, 50, 2), meta=np.ndarray>
    call/genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(250, 50, 2), meta=np.ndarray>
    variant/id          (variants) <U1 dask.array<chunksize=(250,), meta=np.ndarray>
    sample/family_id    (samples) <U1 dask.array<chunksize=(50,), meta=np.ndarray>
    sample/paternal_id  (samples) <U1 dask.array<chunksize=(50,), meta=np.ndarray>
    sample/maternal_id  (samples) <U1 dask.array<chunksize=(50,), meta=np.ndarray>
    sample/sex          (samples) int8 dask.array<chunksize=(50,), meta=np.ndarray>
    sample/phenotype    (samples) int8 dask.array<chunksize=(50,), meta=np.ndarray>
Attributes:
    contigs:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

### Apply REGENIE

In [6]:
df_cov = test_regenie.load_covariates(dataset_dir)
df_cov.head()

,X000,X001,X002
sample_id,,,
S0000001,1.403492,0.187771,1.146020
S0000002,1.854252,1.652777,-0.902967
S0000003,0.634048,-0.362843,0.012637
S0000004,0.124066,-0.067924,1.644158
S0000005,0.455337,-0.090256,0.585722


In [7]:
df_trait = test_regenie.load_traits(dataset_dir)
df_trait.head()

,Y0000
sample_id,
S0000001,-1.030415
S0000002,0.405862
S0000003,1.107564
S0000004,1.018582
S0000005,0.121986


In [8]:
contigs = ds['variant/contig'].values
G = ds['call/genotype'].sum(dim='ploidy').values
X = df_cov.values
Y = df_trait.values
G.shape, X.shape, Y.shape, contigs.shape

((250, 50), (50, 3), (50, 1), (250,))

In [9]:
res = regenie.regenie_transform(
    G.T, X, Y, 
    contigs, 
    variant_block_size=ps_config['variant_block_size'], 
    sample_block_size=ps_config['sample_block_size'], 
    normalize=True, 
    alphas=ps_config['alphas'],
    orthogonalize=False,
    _glow_adj_dof=True,
    _glow_adj_alpha=True,
    _glow_adj_scaling=True
)

In [10]:
res

<xarray.Dataset>
Dimensions:          (alphas: 1, blocks: 30, contigs: 10, outcomes: 1, samples: 50)
Dimensions without coordinates: alphas, blocks, contigs, outcomes, samples
Data variables:
    base_prediction  (blocks, alphas, samples, outcomes) float64 dask.array<chunksize=(1, 1, 10, 1), meta=np.ndarray>
    meta_prediction  (samples, outcomes) float64 dask.array<chunksize=(10, 1), meta=np.ndarray>
    loco_prediction  (contigs, samples, outcomes) float64 dask.array<chunksize=(1, 10, 1), meta=np.ndarray>

### R2 Score

In [11]:
pairs = [
    ([1, 1], [1, 2]),
    ([1, 0], [1, 2]),
    ([1, -1, 3], [1, 2, 3]),
    ([0, -1, 2], [1, 2, 3]),
    ([3, 2, 1], [1, 2, 3]),
    ([0, 0, 0], [1, 2, 3]),
    ([1.1, 2.1, 3.1], [1, 2, 3]),
    ([1.1, 1.9, 3.0], [1, 2, 3]),
    ([1, 2, 3], [1, 2, 3]),
    ([1, 1, 1], [1, 1, 1]),
    ([1, 1, 1], [1, 2, 3]),
    ([1, 2, 3], [1, 1, 1]),
]
res = []
for pair in pairs:
    yp, yt = pair
    yp, yt = np.array(yp), np.array(yt)
    res.append((list(yp), list(yt), sk_r2(yt, yp)))#, sg_r2(yp, yt)))
print('\n'.join(str(t) + ',' for t in res))

([1, 1], [1, 2], -1.0),
([1, 0], [1, 2], -7.0),
([1, -1, 3], [1, 2, 3], -3.5),
([0, -1, 2], [1, 2, 3], -4.5),
([3, 2, 1], [1, 2, 3], -3.0),
([0, 0, 0], [1, 2, 3], -6.0),
([1.1, 2.1, 3.1], [1, 2, 3], 0.985),
([1.1, 1.9, 3.0], [1, 2, 3], 0.99),
([1, 2, 3], [1, 2, 3], 1.0),
([1, 1, 1], [1, 1, 1], 1.0),
([1, 1, 1], [1, 2, 3], -1.5),
([1, 2, 3], [1, 1, 1], 0.0),
